# Reservas Provadas de Petróleo e Gás Natural

In [25]:
import pandas as pd
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.2")
df.drop(df.head(4).index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.drop(df.tail(12).index, inplace=True)
df.drop(columns=df.columns[0], axis=1, inplace=True)
new_header = df.iloc[0]
new_header.iloc[0] = "ANO"
df = df[1:]
df.columns = new_header
df.dropna(inplace=True)
df.rename(columns={
    'ANO': 'ano',
    'PETRÓLEO': 'petróleo_10^3m3',
    'GÁS NATURAL': 'gás_natural_10^6m3'
}, inplace=True)
reservas_petroleo_gas = df

# Recursos Hidráulicos

In [26]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.3")
df.drop(df.head(5).index, inplace=True)
df.drop(df.tail(1).index, inplace=True)
df.drop(columns=df.columns[0], axis=1, inplace=True)
new_header = df.iloc[0]
new_header.iloc[0] = "ANO"
df = df[1:]
df.columns = new_header
df.rename(columns = {
    'INVENTARIADO + APROVEITADO / INVENTORIED': "inventariado_mw",
    'ESTIMADO / ESTIMATED\n': "estimado_mw",
    'TOTAL': 'total_mw'
}, inplace=True)
df.reset_index(inplace=True, drop=True)
df.dropna(inplace=True)
recursos_hidraulicos = df

# Reservas de Carvão Mineral

In [27]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.4")
df.drop(columns=df.columns[0], axis=1, inplace=True)
df.drop(df.head(5).index, inplace=True)
new_header = df.iloc[0]
new_header.iloc[0] = "ANO"
df = df[1:]
df.columns = new_header
df.drop(df.head(1).index, inplace=True)
df.rename(columns=lambda x: x.lower(), inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
reservas_carvao_mineral = df


# Reservas de Urânio

In [28]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="tabela 6.5")
df.drop(df.head(6).index, inplace=True)
df.drop(columns=df.columns[0], inplace=True)
new_header = df.iloc[0]
df = df[1:]
new_header.iloc[0] = "ANO"
df.columns = new_header
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.rename(columns = {
    'U3O8': "ton_U3O8"
})
reservas_de_uranio = df

# Recursos e Reservas energéticas brasileiras

In [29]:
df = pd.read_excel('../databases/Dados EPE/Recursos e Reservas Energéticas.xlsx', sheet_name="Tabela 6.1")
df.drop(df.head(3).index, inplace=True)
df.dropna(how='all', inplace=True, axis=0)
df.reset_index(inplace=True,drop=True)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df.drop(index=df.index[6:23], inplace=True)
# df.drop(df.tail(6).index, inplace=True) ===== Isso parece remover petróleo e gás, isso foi intencional?
df.drop(df.head(2).index, inplace=True)
df = df.iloc[:, :-2]
df.dropna(inplace=True)
df = df.set_axis(['energia', 'unidade_de_medida', 'medidas', 'inferidas', 'total', 'equivalência'], axis=1)
df['energia'] = df['energia'].apply(lambda x: x.split(' ')[0])
df.reset_index(inplace=True, drop=True)
recursos_reservas_energeticas_br = df


In [30]:
dict_recursos_reservas = {
    'RESERVAS_PROVADAS_DE_PETROLEO_E_GAS_NATURAL': reservas_petroleo_gas,
    'RECURSOS_HIDRAULICOS': recursos_hidraulicos,
    'RESERVAS_DE_CARVÃO_MINERAL': reservas_carvao_mineral,
    'RESERVAS_DE_URANIO': reservas_de_uranio,
    'RECURSOS_E_RESERVAS_ENERGÉTICAS_BRASILEIRAS': recursos_reservas_energeticas_br
}

In [31]:
dict_recursos_reservas['RESERVAS_DE_URANIO']

,ANO,NaN
0,RESERVAS DE URÂNIO 1,ton
1,1973,6292
2,1974,11040
3,1975,11041
4,1976,26380
5,1977,66800
6,1978,142300
7,1979,215300
8,1980,236300
9,1981,266300
